In [3]:

from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import IntegerType
from pyspark import StorageLevel

spark = (
    SparkSession.builder
    .appName("Phase1_Ingestion_Cleaning")
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY")
    .getOrCreate()
)
spark.sparkContext.setLogLevel("WARN")

INPUT_CSV = "/path/to/traffic_data_large.csv"

In [5]:

from pathlib import Path

def find_file(filename="traffic_data_large.csv", search_roots=("/", )):
    matches = []
    for root in search_roots:
        root_path = Path(root)
        if root_path.exists():
            for p in root_path.rglob(filename):
                # Skip system, proc, dev, etc. for speed
                if any(str(p).startswith(s) for s in ["/proc", "/sys", "/dev", "/run", "/snap"]):
                    continue
                matches.append(str(p))
                if len(matches) >= 5:
                    break
    return matches

candidates = find_file("traffic_data_large.csv", search_roots=(".", "/content", "/home", "/Workspace", "/dbfs", "/FileStore"))
print("Candidate paths:", candidates)




Candidate paths: ['traffic_data_large.csv', '/content/traffic_data_large.csv']


In [7]:

INPUT_CSV = "/content/traffic_data_large.csv"


In [9]:

import os, pathlib
print("INPUT_CSV =", INPUT_CSV)
print("Absolute:", os.path.isabs(INPUT_CSV))
print("Exists (Python):", os.path.exists(INPUT_CSV))



INPUT_CSV = /content/traffic_data_large.csv
Absolute: True
Exists (Python): True


In [10]:

from pyspark.sql.utils import AnalysisException

try:
    _test_df = (spark.read
        .option("header", True)
        .option("inferSchema", "false")
        .csv(INPUT_CSV)
    )
    print("Spark can read INPUT_CSV directly ")
except AnalysisException as e:
    print("Direct read failed. Trying file:// URI …")

    if not INPUT_CSV.startswith("dbfs:") and not INPUT_CSV.startswith("hdfs:") and not INPUT_CSV.startswith("s3:"):
        uri = "file://" + INPUT_CSV if INPUT_CSV.startswith("/") else "file://" + str(pathlib.Path.cwd() / INPUT_CSV)
        print("Trying URI:", uri)
        _test_df = (spark.read
            .option("header", True)
            .option("inferSchema", "false")
            .csv(uri)
        )
        INPUT_CSV = uri
        print("Spark can read via file:// ")


Spark can read INPUT_CSV directly ✅


In [12]:

#  Phase 1:
df_raw = (
    spark.read
    .option("header", True)
    .option("inferSchema", "false")
    .csv(INPUT_CSV)
)


for c in df_raw.columns:
    df_raw = df_raw.withColumn(c, F.trim(F.col(c)))

print("Schema (raw):")
df_raw.printSchema()
print("Row count (raw):", df_raw.count())
df_raw.show(10, truncate=False)



Schema (raw):
root
 |-- sensor_id: string (nullable = true)
 |-- location: string (nullable = true)
 |-- road_name: string (nullable = true)
 |-- vehicle_count: string (nullable = true)
 |-- avg_speed: string (nullable = true)
 |-- temperature: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- status: string (nullable = true)

Row count (raw): 500000
+---------+---------+---------------+-------------+---------+-----------+-------------------+--------+
|sensor_id|location |road_name      |vehicle_count|avg_speed|temperature|timestamp          |status  |
+---------+---------+---------------+-------------+---------+-----------+-------------------+--------+
|S105     |Chennai  |OMR            |invalid      |NULL     |39         |12/01/2026 06:00:00|INACTIVE|
|S113     |Chennai  |Mount Road     |103          |73.5     |36         |2026-01-12 06:00:05|ACTIVE  |
|S228     |Delhi    |Janpath        |16           |20.0     |35         |2026-01-12 06:00:10|ACTIVE  |
|S160   

In [ ]:

from pyspark.sql.types import IntegerType

# Preserve original timestamp for audit
df = df_raw.withColumn("timestamp_original", F.col("timestamp"))

# vehicle_count → vehicle_count_clean
df = (df
    .withColumn("vc_lower", F.lower(F.col("vehicle_count")))
    .withColumn(
        "vehicle_count_clean",
        F.when(
            F.col("vc_lower").isNull() | (F.col("vc_lower") == "") | (F.col("vc_lower") == "invalid"),
            F.lit(None).cast("int")
        ).when(
            F.col("vc_lower").rlike(r"^\\d+$"),
            F.col("vc_lower").cast(IntegerType())
        ).otherwise(F.lit(None).cast("int"))
    )
    .drop("vc_lower")
)

# avg_speed → avg_speed_clean
df = (df
    .withColumn("as_lower", F.lower(F.col("avg_speed")))
    .withColumn(
        "avg_speed_clean",
        F.when(
            F.col("as_lower").isNull() | (F.col("as_lower") == ""),
            F.lit(None).cast("double")
        ).otherwise(F.col("as_lower").cast("double"))
    )
    .drop("as_lower")
)

# timestamp → event_time (three formats)
ts1 = F.to_timestamp("timestamp", "yyyy-MM-dd HH:mm:ss")
ts2 = F.to_timestamp("timestamp", "dd/MM/yyyy HH:mm:ss")
ts3 = F.to_timestamp("timestamp", "yyyy/MM/dd HH:mm:ss")
df = df.withColumn("event_time", F.coalesce(ts1, ts2, ts3))

df.select("sensor_id","location","road_name","vehicle_count","vehicle_count_clean",
          "avg_speed","avg_speed_clean","timestamp","event_time").show(10, truncate=False)


PHASE 2

In [13]:

# 1 Trim all string columns
from pyspark.sql import functions as F

df_trimmed = df_raw
for c in df_trimmed.columns:
    df_trimmed = df_trimmed.withColumn(c, F.trim(F.col(c)))

df_trimmed.printSchema()
df_trimmed.show(5, truncate=False)


root
 |-- sensor_id: string (nullable = true)
 |-- location: string (nullable = true)
 |-- road_name: string (nullable = true)
 |-- vehicle_count: string (nullable = true)
 |-- avg_speed: string (nullable = true)
 |-- temperature: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- status: string (nullable = true)

+---------+---------+---------------+-------------+---------+-----------+-------------------+--------+
|sensor_id|location |road_name      |vehicle_count|avg_speed|temperature|timestamp          |status  |
+---------+---------+---------------+-------------+---------+-----------+-------------------+--------+
|S105     |Chennai  |OMR            |invalid      |NULL     |39         |12/01/2026 06:00:00|INACTIVE|
|S113     |Chennai  |Mount Road     |103          |73.5     |36         |2026-01-12 06:00:05|ACTIVE  |
|S228     |Delhi    |Janpath        |16           |20.0     |35         |2026-01-12 06:00:10|ACTIVE  |
|S160     |Bangalore|MG Road        |27       

In [14]:

# 2 Clean vehicle_count
from pyspark.sql.types import IntegerType

df_vc = (
    df_trimmed

    .withColumn("vehicle_count_lower", F.lower(F.col("vehicle_count")))

    .withColumn(
        "vehicle_count_clean",
        F.when(
            F.col("vehicle_count_lower").isNull() |
            (F.col("vehicle_count_lower") == "") |
            (F.col("vehicle_count_lower") == "invalid"),
            F.lit(None).cast(IntegerType())
        )
        .when(
            F.col("vehicle_count_lower").rlike(r"^\d+$"),
            F.col("vehicle_count_lower").cast(IntegerType())
        )
        .otherwise(F.lit(None).cast(IntegerType()))
    )
    .drop("vehicle_count_lower")
)

df_vc.select("vehicle_count", "vehicle_count_clean").show(10, truncate=False)


+-------------+-------------------+
|vehicle_count|vehicle_count_clean|
+-------------+-------------------+
|invalid      |NULL               |
|103          |103                |
|16           |16                 |
|27           |27                 |
|115          |115                |
|13           |13                 |
|81           |81                 |
|40           |40                 |
|37           |37                 |
|89           |89                 |
+-------------+-------------------+
only showing top 10 rows


In [15]:

# 3 Clean avg_speed: empty -> null ; cast to DoubleType
df_speed = (
    df_vc
    .withColumn("avg_speed_lower", F.lower(F.col("avg_speed")))
    .withColumn(
        "avg_speed_clean",
        F.when(
            F.col("avg_speed_lower").isNull() |
            (F.col("avg_speed_lower") == ""),
            F.lit(None).cast("double")
        )
        .otherwise(F.col("avg_speed_lower").cast("double"))
    )
    .drop("avg_speed_lower")
)

df_speed.select("avg_speed", "avg_speed_clean").show(10, truncate=False)



+---------+---------------+
|avg_speed|avg_speed_clean|
+---------+---------------+
|NULL     |NULL           |
|73.5     |73.5           |
|20.0     |20.0           |
|27.1     |27.1           |
|59.3     |59.3           |
|23.6     |23.6           |
|71.1     |71.1           |
|36.3     |36.3           |
|49.3     |49.3           |
|79.3     |79.3           |
+---------+---------------+
only showing top 10 rows


In [19]:

df_clean_phase2 = df_time.withColumn("timestamp_original", F.col("timestamp"))



In [ ]:

# 4 Parse timestamp into event_time with multiple formats
ts1 = F.to_timestamp("timestamp", "yyyy-MM-dd HH:mm:ss")
ts2 = F.to_timestamp("timestamp", "dd/MM/yyyy HH:mm:ss")
ts3 = F.to_timestamp("timestamp", "yyyy/MM/dd HH:mm:ss")

df_time = df_speed.withColumn("event_time", F.coalesce(ts1, ts2, ts3))

df_time.select("timestamp", "event_time").show(10, truncate=False)


In [22]:

df_time = df_speed.withColumn(
    "timestamp_norm",
    F.when(F.lower(F.col("timestamp")) == "invalid_time", None).otherwise(F.col("timestamp"))
)
ts1 = F.to_timestamp("timestamp_norm", "yyyy-MM-dd HH:mm:ss")
ts2 = F.to_timestamp("timestamp_norm", "dd/MM/yyyy HH:mm:ss")
ts3 = F.to_timestamp("timestamp_norm", "yyyy/MM/dd HH:mm:ss")
df_time = df_time.withColumn("event_time", F.coalesce(ts1, ts2, ts3)).drop("timestamp_norm")


In [23]:

df_clean_phase2 = df_time.withColumn("timestamp_original", F.col("timestamp"))


phase 3

In [24]:

# t1 Count invalid vehicle_count rows
invalid_vehicle_count_raw = df_time.where(
    F.col("vehicle_count_clean").isNull()
).count()

print("Invalid vehicle_count rows (RAW):", invalid_vehicle_count_raw)


Invalid vehicle_count rows (RAW): 49873


In [26]:

df_temp = df_speed.drop("event_time")


In [ ]:

# 3.2 Count invalid timestamp rows (rows where event_time is NULL)
invalid_timestamp_rows = df_time.where(
    F.col("event_time").isNull()
).count()

print("Invalid timestamp rows (RAW):", invalid_timestamp_rows)


In [30]:
del df_time

In [31]:
df_base = df_trimmed

In [32]:

from pyspark.sql.types import IntegerType

df_clean = (
    df_base
    .withColumn("vehicle_count_lower", F.lower(F.col("vehicle_count")))
    .withColumn(
        "vehicle_count_clean",
        F.when(
            F.col("vehicle_count_lower").isNull() |
            (F.col("vehicle_count_lower") == "") |
            (F.col("vehicle_count_lower") == "invalid"),
            None
        ).when(
            F.col("vehicle_count_lower").rlike("^[0-9]+$"),
            F.col("vehicle_count_lower").cast(IntegerType())
        ).otherwise(None)
    )
    .drop("vehicle_count_lower")
    .withColumn("avg_speed_lower", F.lower(F.col("avg_speed")))
    .withColumn(
        "avg_speed_clean",
        F.when(F.col("avg_speed_lower").isNull() | (F.col("avg_speed_lower") == ""), None)
         .otherwise(F.col("avg_speed_lower").cast("double"))
    )
    .drop("avg_speed_lower")
)


In [ ]:

df_clean = (
    df_clean
    .withColumn(
        "timestamp_norm",
        F.when(F.lower(F.col("timestamp")) == "invalid_time", None)
         .when(F.col("timestamp") == "", None)
         .otherwise(F.col("timestamp"))
    )
    .withColumn("ts1", F.try_to_timestamp("timestamp_norm", "yyyy-MM-dd HH:mm:ss"))
    .withColumn("ts2", F.try_to_timestamp("timestamp_norm", "dd/MM/yyyy HH:mm:ss"))
    .withColumn("ts3", F.try_to_timestamp("timestamp_norm", "yyyy/MM/dd HH:mm:ss"))
    .withColumn("ts4", F.try_to_timestamp("timestamp_norm"))
    .withColumn("event_time", F.coalesce("ts1", "ts2", "ts3", "ts4"))
    .drop("ts1", "ts2", "ts3", "ts4")
)


In [34]:
df_clean.select("timestamp", "timestamp_norm", "event_time").show(20, truncate=False)

{"ts": "2026-01-19 09:51:06.553", "level": "ERROR", "logger": "DataFrameQueryContextLogger", "msg": "[UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `timestamp_norm` cannot be resolved. Did you mean one of the following? [`timestamp`, `status`, `location`, `road_name`, `sensor_id`]. SQLSTATE: 42703", "context": {"file": "jdk.internal.reflect.GeneratedMethodAccessor31.invoke(Unknown Source)", "line": "", "fragment": "col", "errorClass": "UNRESOLVED_COLUMN.WITH_SUGGESTION"}, "exception": {"class": "Py4JJavaError", "msg": "An error occurred while calling o505.select.\n: org.apache.spark.sql.AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `timestamp_norm` cannot be resolved. Did you mean one of the following? [`timestamp`, `status`, `location`, `road_name`, `sensor_id`]. SQLSTATE: 42703;\n'Project [timestamp#110, 'timestamp_norm, 'event_time]\n+- Project [sensor_id#104, location#105, road_nam

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `timestamp_norm` cannot be resolved. Did you mean one of the following? [`timestamp`, `status`, `location`, `road_name`, `sensor_id`]. SQLSTATE: 42703;
'Project [timestamp#110, 'timestamp_norm, 'event_time]
+- Project [sensor_id#104, location#105, road_name#106, vehicle_count#107, avg_speed#108, temperature#109, timestamp#110, status#111, vehicle_count_clean#251, avg_speed_clean#253]
   +- Project [sensor_id#104, location#105, road_name#106, vehicle_count#107, avg_speed#108, temperature#109, timestamp#110, status#111, vehicle_count_clean#251, avg_speed_lower#252, CASE WHEN (isnull(avg_speed_lower#252) OR (avg_speed_lower#252 = )) THEN cast(null as double) ELSE cast(avg_speed_lower#252 as double) END AS avg_speed_clean#253]
      +- Project [sensor_id#104, location#105, road_name#106, vehicle_count#107, avg_speed#108, temperature#109, timestamp#110, status#111, vehicle_count_clean#

In [ ]:
#t2
invalid_timestamp_rows = df_clean.where(F.col("event_time").isNull()).count()
print("Invalid timestamp rows (RAW):", invalid_timestamp_rows)


In [39]:
#t3
df_active = df_clean.where(F.upper(F.col("status")) == "ACTIVE")
print("ACTIVE rows:", df_active.count())


ACTIVE rows: 475000


In [ ]:
#t4
active_invalid_vehicle_count = df_active.where(F.col("vehicle_count_clean").isNull()).count()
active_invalid_timestamp = df_active.where(F.col("event_time").isNull()).count()
active_missing_speed = df_active.where(F.col("avg_speed_clean").isNull()).count()

print("=== ACTIVE DATA QUALITY REPORT ===")
print("Invalid vehicle_count:", active_invalid_vehicle_count)
print("Invalid timestamp:", active_invalid_timestamp)
print("Missing avg_speed:", active_missing_speed)


phase 4

In [ ]:

cleaned = df_active.select(
    "sensor_id","location","road_name",
    "vehicle_count_clean","avg_speed_clean",
    "event_time","timestamp_original","status"
)


In [ ]:

# t1 Average speed per location
avg_speed_per_location = (
    cleaned.groupBy("location")
           .agg(F.avg("avg_speed_clean").alias("avg_speed_location"))
           .orderBy(F.col("avg_speed_location").asc_nulls_last())
)

avg_speed_per_location.show(truncate=False)


In [ ]:

# t2 Total vehicle count per road
total_vehicle_per_road = (
    cleaned.groupBy("road_name")
           .agg(F.sum("vehicle_count_clean").alias("total_vehicle_count"))
           .orderBy(F.col("total_vehicle_count").desc_nulls_last())
)

total_vehicle_per_road.show(truncate=False)


In [ ]:

# t3 Peak traffic time per location
hourly = (
    cleaned.withColumn("event_hour", F.date_trunc("hour", F.col("event_time")))
)

per_loc_hour = (
    hourly.groupBy("location", "event_hour")
          .agg(F.sum("vehicle_count_clean").alias("vehicles"))
)

from pyspark.sql import Window
w_loc = Window.partitionBy("location").orderBy(F.col("vehicles").desc_nulls_last())

peak_time_per_location = (
    per_loc_hour.withColumn("rank", F.row_number().over(w_loc))
                .where(F.col("rank") == 1)
                .drop("rank")
                .orderBy("location")
)

peak_time_per_location.show(truncate=False)


In [ ]:

# t4 Roads with lowest avg speed (most congested)
road_speed = (
    cleaned.groupBy("road_name")
           .agg(F.avg("avg_speed_clean").alias("avg_speed_road"))
           .orderBy(F.col("avg_speed_road").asc_nulls_last())
)

road_speed.show(50, truncate=False)


In [ ]:
#ohase 5 task1
from pyspark.sql import functions as F
from pyspark.sql import Window
road_speed = (
    cleaned.groupBy("road_name")
           .agg(F.avg("avg_speed_clean").alias("avg_speed_road"))
)

w_congestion = Window.orderBy(F.col("avg_speed_road").asc_nulls_last())
road_speed_ranked = (
    road_speed
    .withColumn("congestion_rank", F.dense_rank().over(w_congestion))
    .orderBy(F.col("congestion_rank").asc(), F.col("avg_speed_road").asc_nulls_last(), F.col("road_name").asc())
)

road_speed_ranked.show(50, truncate=False)



In [ ]:

#t2
loc_road_volume = (
    cleaned.groupBy("location", "road_name")
           .agg(F.sum("vehicle_count_clean").alias("vehicles"))
)


w_loc_road = Window.partitionBy("location").orderBy(F.col("vehicles").desc_nulls_last())
loc_road_ranked = (
    loc_road_volume
    .withColumn("volume_rank_in_location", F.dense_rank().over(w_loc_road))
    .orderBy("location", "volume_rank_in_location", F.col("vehicles").desc_nulls_last(), "road_name")
)

loc_road_ranked.show(50, truncate=False)



In [ ]:

#t3
loc_road_speed = (
    cleaned.groupBy("location", "road_name")
           .agg(F.avg("avg_speed_clean").alias("avg_speed_loc_road"))
)

w_loc_congestion = Window.partitionBy("location").orderBy(F.col("avg_speed_loc_road").asc_nulls_last())

top3_congested_per_location = (
    loc_road_speed
    .withColumn("congestion_rank_in_location", F.dense_rank().over(w_loc_congestion))
    .where(F.col("congestion_rank_in_location") <= 3)
    .orderBy("location", "congestion_rank_in_location", "road_name")
)

top3_congested_per_location.show(100, truncate=False)


phase 6

In [49]:
#t1


w = Window.partitionBy("sensor_id").orderBy("event_time")



In [61]:
#t2
count_spike = (
    cleaned
    .withColumn("prev_count", F.lag("vehicle_count_clean").over(w))
    .withColumn("count_spike", F.col("vehicle_count_clean") - F.col("prev_count"))
    .where(F.col("count_spike") >= 40)
)

count_spike.select("sensor_id","event_time","prev_count","vehicle_count_clean","count_spike").show(20, False)


+---------+-------------------+----------+-------------------+-----------+
|sensor_id|event_time         |prev_count|vehicle_count_clean|count_spike|
+---------+-------------------+----------+-------------------+-----------+
|S101     |NULL               |25        |109                |84         |
|S101     |NULL               |30        |111                |81         |
|S101     |NULL               |36        |99                 |63         |
|S101     |NULL               |15        |119                |104        |
|S101     |NULL               |13        |114                |101        |
|S101     |NULL               |25        |76                 |51         |
|S101     |2026-01-12 09:25:30|22        |116                |94         |
|S101     |2026-01-12 09:36:20|46        |91                 |45         |
|S101     |2026-01-12 10:57:40|26        |92                 |66         |
|S101     |2026-01-12 11:59:10|16        |89                 |73         |
|S101     |2026-01-12 13:

In [54]:

df_clean.printSchema()
df_clean.columns


root
 |-- sensor_id: string (nullable = true)
 |-- location: string (nullable = true)
 |-- road_name: string (nullable = true)
 |-- vehicle_count: string (nullable = true)
 |-- avg_speed: string (nullable = true)
 |-- temperature: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- status: string (nullable = true)
 |-- vehicle_count_clean: integer (nullable = true)
 |-- avg_speed_clean: double (nullable = true)



['sensor_id',
 'location',
 'road_name',
 'vehicle_count',
 'avg_speed',
 'temperature',
 'timestamp',
 'status',
 'vehicle_count_clean',
 'avg_speed_clean']

In [57]:

import datetime
from pyspark.sql import functions as F
from pyspark.sql.types import TimestampType

def parse_any_timestamp(s):
    if s is None:
        return None
    s = s.strip()
    if s == "" or s.lower() == "invalid_time":
        return None

    fmts = [
        "%Y-%m-%d %H:%M:%S",
        "%d/%m/%Y %H:%M:%S",
        "%Y/%m/%d %H:%M:%S"
    ]

    for fmt in fmts:
        try:
            return datetime.datetime.strptime(s, fmt)
        except:
            pass

    # final fallback
    try:
        return datetime.datetime.fromisoformat(s)
    except:
        return None

parse_ts_udf = F.udf(parse_any_timestamp, TimestampType())


In [58]:
df_time = df_clean.withColumn("event_time", parse_ts_udf(F.col("timestamp")))

In [59]:
df_time.select("timestamp", "event_time").show(10, False)

+-------------------+-------------------+
|timestamp          |event_time         |
+-------------------+-------------------+
|12/01/2026 06:00:00|2026-01-12 06:00:00|
|2026-01-12 06:00:05|2026-01-12 06:00:05|
|2026-01-12 06:00:10|2026-01-12 06:00:10|
|2026-01-12 06:00:15|2026-01-12 06:00:15|
|2026-01-12 06:00:20|2026-01-12 06:00:20|
|2026-01-12 06:00:25|2026-01-12 06:00:25|
|2026-01-12 06:00:30|2026-01-12 06:00:30|
|2026-01-12 06:00:35|2026-01-12 06:00:35|
|2026-01-12 06:00:40|2026-01-12 06:00:40|
|2026-01-12 06:00:45|2026-01-12 06:00:45|
+-------------------+-------------------+
only showing top 10 rows


In [60]:

cleaned = df_time.select(
    "sensor_id", "location", "road_name",
    "vehicle_count_clean", "avg_speed_clean",
    "event_time", "timestamp", "status"
)


PHASE 7

In [62]:

# T1 Partitions before any optimization
print("Partitions BEFORE:", cleaned.rdd.getNumPartitions())



Partitions BEFORE: 2


In [63]:

# T2 Execution plan before optimization
cleaned.explain(False)


== Physical Plan ==
*(2) Project [sensor_id#104, location#105, road_name#106, vehicle_count_clean#251, CASE WHEN (isnull(avg_speed_lower#252) OR (avg_speed_lower#252 = )) THEN null ELSE cast(avg_speed_lower#252 as double) END AS avg_speed_clean#253, pythonUDF0#327 AS event_time#295, timestamp#110, status#111]
+- BatchEvalPython [parse_any_timestamp(timestamp#110)#294], [pythonUDF0#327]
   +- *(1) Project [sensor_id#104, location#105, road_name#106, timestamp#110, status#111, CASE WHEN ((isnull(vehicle_count_lower#250) OR (vehicle_count_lower#250 = )) OR (vehicle_count_lower#250 = invalid)) THEN null WHEN RLIKE(vehicle_count_lower#250, ^[0-9]+$) THEN cast(vehicle_count_lower#250 as int) END AS vehicle_count_clean#251, lower(avg_speed#108) AS avg_speed_lower#252]
      +- *(1) Project [trim(trim(sensor_id#42, None), None) AS sensor_id#104, trim(trim(location#43, None), None) AS location#105, trim(trim(road_name#44, None), None) AS road_name#106, trim(trim(avg_speed#46, None), None) AS av

In [64]:

# T3 Repartition using a frequently-used column
cleaned_part = cleaned.repartition("location")


In [65]:

# T4 Cache and materialize
cleaned_part = cleaned_part.cache()
cleaned_part.count()


500000

In [66]:

# T5 Partitions after repartition("location")
print("Partitions AFTER:", cleaned_part.rdd.getNumPartitions())


Partitions AFTER: 200


In [67]:

# T6 Execution plan after optimization
cleaned_part.explain(False)


== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=true
+- == Final Plan ==
   ResultQueryStage 1
   +- TableCacheQueryStage 0
      +- InMemoryTableScan [sensor_id#104, location#105, road_name#106, vehicle_count_clean#251, avg_speed_clean#253, event_time#295, timestamp#110, status#111]
            +- InMemoryRelation [sensor_id#104, location#105, road_name#106, vehicle_count_clean#251, avg_speed_clean#253, event_time#295, timestamp#110, status#111], StorageLevel(disk, memory, deserialized, 1 replicas)
                  +- AdaptiveSparkPlan isFinalPlan=true
                  +- == Final Plan ==
                     ResultQueryStage 1
                     +- ShuffleQueryStage 0
                        +- Exchange hashpartitioning(location#105, 200), REPARTITION_BY_COL, [plan_id=622]
                           +- *(2) Project [sensor_id#104, location#105, road_name#106, vehicle_count_clean#251, CASE WHEN (isnull(avg_speed_lower#252) OR (avg_speed_lower#252 = )) THEN null ELSE cast(avg_spe

PHASE 8

In [68]:

# T1 Convert DataFrame to RDD
rdd = cleaned_part.select("vehicle_count_clean", "location").rdd


In [69]:

# T2 Total vehicle count using reduce (null-safe)
total_vehicle_rdd = (
    rdd.map(lambda row: row[0] if row[0] is not None else 0)
       .reduce(lambda a, b: a + b)
)

print("Total vehicle count via RDD.reduce:", total_vehicle_rdd)


Total vehicle count via RDD.reduce: 29255173


In [70]:

# T3 Count records per location
records_per_location_rdd = (
    rdd.map(lambda row: (row[1], 1))       # (location, 1)
       .reduceByKey(lambda a, b: a + b)   # sum counts
)

for x in records_per_location_rdd.take(10):
    print(x)


('Mumbai', 71645)
('Delhi', 71653)
('Kolkata', 71591)
('Bangalore', 71285)
('Pune', 70842)
('Chennai', 71326)
('Hyderabad', 71658)


In [71]:

# T4 Convert the reduceByKey output to a DataFrame
df_records_per_location = records_per_location_rdd.toDF(["location", "record_count"])

df_records_per_location.show(truncate=False)


+---------+------------+
|location |record_count|
+---------+------------+
|Mumbai   |71645       |
|Delhi    |71653       |
|Kolkata  |71591       |
|Bangalore|71285       |
|Pune     |70842       |
|Chennai  |71326       |
|Hyderabad|71658       |
+---------+------------+



In [72]:

# T5 Filter: keep rows where count > 50
high_count_rdd = rdd.filter(lambda row: row[0] is not None and row[0] > 50)

print("Sample high count rows:", high_count_rdd.take(10))


Sample high count rows: [Row(vehicle_count_clean=87, location='Bangalore'), Row(vehicle_count_clean=68, location='Bangalore'), Row(vehicle_count_clean=74, location='Bangalore'), Row(vehicle_count_clean=74, location='Bangalore'), Row(vehicle_count_clean=100, location='Bangalore'), Row(vehicle_count_clean=85, location='Bangalore'), Row(vehicle_count_clean=82, location='Bangalore'), Row(vehicle_count_clean=59, location='Bangalore'), Row(vehicle_count_clean=119, location='Bangalore'), Row(vehicle_count_clean=117, location='Bangalore')]


PHASE 9

In [73]:

# 1 Sort roads by lowest avg speed (congestion metric)
roads_by_congestion = (
    cleaned.groupBy("road_name")
           .agg(F.avg("avg_speed_clean").alias("avg_speed_road"))
           .orderBy(F.col("avg_speed_road").asc_nulls_last())
)

roads_by_congestion.show(20, truncate=False)


+---------------+------------------+
|road_name      |avg_speed_road    |
+---------------+------------------+
|EM Bypass      |47.29303486907817 |
|Link Road      |47.368898290300194|
|FC Road        |47.37180375306915 |
|Nagar Rd       |47.40516559782131 |
|Whitefield Rd  |47.414129958840526|
|MG Road        |47.421224489796046|
|Howrah Rd      |47.4229107618311  |
|Gachibowli Rd  |47.42927475774559 |
|GST Road       |47.43261706083269 |
|Hitech City Rd |47.483875274559495|
|University Rd  |47.49609046283315 |
|Outer Ring Rd  |47.49839820874945 |
|Eastern Express|47.51013993541459 |
|Western Express|47.51883443020463 |
|Ring Road      |47.537259072755596|
|OMR            |47.54642794453223 |
|Park Street    |47.603439327967635|
|NH48           |47.62648557609277 |
|Madhapur Rd    |47.634299244070036|
|Janpath        |47.65433455433439 |
+---------------+------------------+
only showing top 20 rows


In [74]:

# 2 Create Sets A and B

# Set A
set_A = cleaned.where(F.col("avg_speed_clean") < 25).select("road_name").distinct()

# Set B
set_B = cleaned.where(F.col("vehicle_count_clean") > 60).select("road_name").distinct()


In [75]:

# 3 Intersection
roads_both = set_A.join(set_B, "road_name", "inner")

roads_both.show(truncate=False)


+---------------+
|road_name      |
+---------------+
|University Rd  |
|Western Express|
|Eastern Express|
|FC Road        |
|Whitefield Rd  |
|Link Road      |
|Outer Ring Rd  |
|Gachibowli Rd  |
|Janpath        |
|Hitech City Rd |
|GST Road       |
|OMR            |
|NH48           |
|Ring Road      |
|Mount Road     |
|Howrah Rd      |
|Park Street    |
|Madhapur Rd    |
|EM Bypass      |
|MG Road        |
+---------------+
only showing top 20 rows


PHASE 10

In [76]:

OUT_DIR = "/content/traffic_outputs_phase10"


In [77]:

# 1 Parquet (partitioned by location)
(
    cleaned
    .write
    .mode("overwrite")
    .partitionBy("location")
    .parquet(f"{OUT_DIR}/cleaned_parquet_by_location")
)
print(" Wrote partitioned Parquet:", f"{OUT_DIR}/cleaned_parquet_by_location")


 Wrote partitioned Parquet: /content/traffic_outputs_phase10/cleaned_parquet_by_location


In [78]:

from pyspark.sql import functions as F, Window

if "road_speed_ranked" not in locals():
    road_speed = (
        cleaned.groupBy("road_name")
               .agg(F.avg("avg_speed_clean").alias("avg_speed_road"))
    )
    w_congestion = Window.orderBy(F.col("avg_speed_road").asc_nulls_last())
    road_speed_ranked = (
        road_speed
        .withColumn("congestion_rank", F.dense_rank().over(w_congestion))
    )

#2
(
    road_speed_ranked
    .write
    .mode("overwrite")
    .orc(f"{OUT_DIR}/congestion_analytics_orc")
)
print(" Wrote ORC analytics:", f"{OUT_DIR}/congestion_analytics_orc")


 Wrote ORC analytics: /content/traffic_outputs_phase10/congestion_analytics_orc


In [79]:

# T3
read_cleaned = spark.read.parquet(f"{OUT_DIR}/cleaned_parquet_by_location")
read_orc     = spark.read.orc(f"{OUT_DIR}/congestion_analytics_orc")

print("Cleaned Parquet row count:", read_cleaned.count(), "| Original cleaned count:", cleaned.count())
print("ORC analytics row count:", read_orc.count(), "| Original analytics count:", road_speed_ranked.count())

print("\n-- Cleaned Parquet schema --")
read_cleaned.printSchema()

print("\n-- ORC analytics schema --")
read_orc.printSchema()

print("\nSample (cleaned Parquet):")
read_cleaned.show(10, truncate=False)

print("\nSample (ORC analytics):")
read_orc.orderBy(F.col("congestion_rank").asc_nulls_last()).show(10, truncate=False)


Cleaned Parquet row count: 500000 | Original cleaned count: 500000
ORC analytics row count: 21 | Original analytics count: 21

-- Cleaned Parquet schema --
root
 |-- sensor_id: string (nullable = true)
 |-- road_name: string (nullable = true)
 |-- vehicle_count_clean: integer (nullable = true)
 |-- avg_speed_clean: double (nullable = true)
 |-- event_time: timestamp (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- status: string (nullable = true)
 |-- location: string (nullable = true)


-- ORC analytics schema --
root
 |-- road_name: string (nullable = true)
 |-- avg_speed_road: double (nullable = true)
 |-- congestion_rank: integer (nullable = true)


Sample (cleaned Parquet):
+---------+---------------+-------------------+---------------+-------------------+-------------------+------+--------+
|sensor_id|road_name      |vehicle_count_clean|avg_speed_clean|event_time         |timestamp          |status|location|
+---------+---------------+-------------------+----------